In [1]:
import segmentation_models as sm
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from fetch_data import fetch
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

2023-02-15 13:17:01.373419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 13:17:01.471188: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-15 13:17:01.473930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-15 13:17:01.473943: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

Segmentation Models: using `tf.keras` framework.


In [2]:
base_dir = '/media/dev/WinD/Curious Dev B/PROJECT STAGE - II/ISIC 2017 classwise segmentation/'
batch_size = 16
input_size = (224, 224)
num_epochs = 5
grp = 'B'
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

callbacks = [ModelCheckpoint('./best_model.h5', 
                             save_weights_only=True, 
                             save_best_only=True, 
                             mode='min'),
            ReduceLROnPlateau(monitor = "val_iou_score",
                             factor = 0.2,
                             patience = 5)]

In [3]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
def denormalize(x):
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

In [4]:
train_dataset, validation_dataset = fetch(base_dir, input_size, grp)

train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=128)

validation_dataset = validation_dataset.batch(1)

2023-02-15 13:17:03.056675: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-15 13:17:03.056834: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-15 13:17:03.056849: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Turbo): /proc/driver/nvidia/version does not exist
2023-02-15 13:17:03.057442: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/dev/anaconda3/lib/python3.9/site-packages/tensorflow/python/dat

In [5]:
model = sm.Unet(backbone_name = "efficientnetb4",
                   input_shape=(224, 224, 3))

model.compile(optimizer=Adam(learning_rate = 0.001), 
              loss='binary_crossentropy', 
              metrics=metrics)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 48  1296        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 112, 112, 48  192         ['stem_conv[0][0]']              
                                )                                                             

                                                                                                  
 block2a_dwconv (DepthwiseConv2  (None, 56, 56, 144)  1296       ['block2a_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block2a_bn (BatchNormalization  (None, 56, 56, 144)  576        ['block2a_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block2a_activation (Activation  (None, 56, 56, 144)  0          ['block2a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block2a_s

 block2c_se_squeeze (GlobalAver  (None, 192)         0           ['block2c_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block2c_se_reshape (Reshape)   (None, 1, 1, 192)    0           ['block2c_se_squeeze[0][0]']     
                                                                                                  
 block2c_se_reduce (Conv2D)     (None, 1, 1, 8)      1544        ['block2c_se_reshape[0][0]']     
                                                                                                  
 block2c_se_expand (Conv2D)     (None, 1, 1, 192)    1728        ['block2c_se_reduce[0][0]']      
                                                                                                  
 block2c_se_excite (Multiply)   (None, 56, 56, 192)  0           ['block2c_activation[0][0]',     
          

 block3a_se_reduce (Conv2D)     (None, 1, 1, 8)      1544        ['block3a_se_reshape[0][0]']     
                                                                                                  
 block3a_se_expand (Conv2D)     (None, 1, 1, 192)    1728        ['block3a_se_reduce[0][0]']      
                                                                                                  
 block3a_se_excite (Multiply)   (None, 28, 28, 192)  0           ['block3a_activation[0][0]',     
                                                                  'block3a_se_expand[0][0]']      
                                                                                                  
 block3a_project_conv (Conv2D)  (None, 28, 28, 56)   10752       ['block3a_se_excite[0][0]']      
                                                                                                  
 block3a_project_bn (BatchNorma  (None, 28, 28, 56)  224         ['block3a_project_conv[0][0]']   
 lization)

 lization)                                                                                        
                                                                                                  
 block3c_drop (FixedDropout)    (None, 28, 28, 56)   0           ['block3c_project_bn[0][0]']     
                                                                                                  
 block3c_add (Add)              (None, 28, 28, 56)   0           ['block3c_drop[0][0]',           
                                                                  'block3b_add[0][0]']            
                                                                                                  
 block3d_expand_conv (Conv2D)   (None, 28, 28, 336)  18816       ['block3c_add[0][0]']            
                                                                                                  
 block3d_expand_bn (BatchNormal  (None, 28, 28, 336)  1344       ['block3d_expand_conv[0][0]']    
 ization) 

...

 D)                                                              ']                               
                                                                                                  
 block6e_bn (BatchNormalization  (None, 7, 7, 1632)  6528        ['block6e_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block6e_activation (Activation  (None, 7, 7, 1632)  0           ['block6e_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block6e_se_squeeze (GlobalAver  (None, 1632)        0           ['block6e_activation[0][0]']     
 agePooling2D)                                                                                    
          

 block6g_activation (Activation  (None, 7, 7, 1632)  0           ['block6g_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block6g_se_squeeze (GlobalAver  (None, 1632)        0           ['block6g_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block6g_se_reshape (Reshape)   (None, 1, 1, 1632)   0           ['block6g_se_squeeze[0][0]']     
                                                                                                  
 block6g_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block6g_se_reshape[0][0]']     
                                                                                                  
 block6g_s

                                                                                                  
 block7a_se_reshape (Reshape)   (None, 1, 1, 1632)   0           ['block7a_se_squeeze[0][0]']     
                                                                                                  
 block7a_se_reduce (Conv2D)     (None, 1, 1, 68)     111044      ['block7a_se_reshape[0][0]']     
                                                                                                  
 block7a_se_expand (Conv2D)     (None, 1, 1, 1632)   112608      ['block7a_se_reduce[0][0]']      
                                                                                                  
 block7a_se_excite (Multiply)   (None, 7, 7, 1632)   0           ['block7a_activation[0][0]',     
                                                                  'block7a_se_expand[0][0]']      
                                                                                                  
 block7a_p

                                                                                                  
 decoder_stage1_upsampling (UpS  (None, 28, 28, 256)  0          ['decoder_stage0b_relu[0][0]']   
 ampling2D)                                                                                       
                                                                                                  
 decoder_stage1_concat (Concate  (None, 28, 28, 592)  0          ['decoder_stage1_upsampling[0][0]
 nate)                                                           ',                               
                                                                  'block4a_expand_activation[0][0]
                                                                 ']                               
                                                                                                  
 decoder_stage1a_conv (Conv2D)  (None, 28, 28, 128)  681984      ['decoder_stage1_concat[0][0]']  
          

                                                                                                  
 decoder_stage4a_relu (Activati  (None, 224, 224, 16  0          ['decoder_stage4a_bn[0][0]']     
 on)                            )                                                                 
                                                                                                  
 decoder_stage4b_conv (Conv2D)  (None, 224, 224, 16  2304        ['decoder_stage4a_relu[0][0]']   
                                )                                                                 
                                                                                                  
 decoder_stage4b_bn (BatchNorma  (None, 224, 224, 16  64         ['decoder_stage4b_conv[0][0]']   
 lization)                      )                                                                 
                                                                                                  
 decoder_s

In [ ]:
history = model.fit(train_dataset, epochs=num_epochs,
                    validation_data=validation_dataset,
                    callbacks=callbacks)

Epoch 1/5


2023-02-15 13:17:18.497103: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 18 of 1024
2023-02-15 13:17:26.325711: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 31 of 1024
2023-02-15 13:17:36.728984: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 39 of 1024
2023-02-15 13:17:46.058083: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 59 of 1024
2023-02-15 13:17:56.648155: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 71 of 1024
2023-02-15 13:18:06.011343: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 76 of 1024
2023-02-15 13:18:16.372132: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer 

72/86 [========================>.....] - ETA: 1:48 - loss: 0.4110 - iou_score: 0.2953 - f1-score: 0.4473

In [ ]:
model.load_weights('best_model.h5') 

In [ ]:
scores = model.evaluate(validation_dataset)

print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric.__name__, value))

In [ ]:
print(tf.data.experimental.cardinality(validation_dataset))
for (img, mask) in validation_dataset:
    print(img.shape)
    print(mask.shape)

In [ ]:
for batch in validation_dataset:
    image, gt_mask = batch
    image = image.numpy()
    gt_mask = gt_mask.numpy()
    pr_mask = model.predict(image).round()
    
    visualize(
        image=denormalize(image.squeeze()),
        gt_mask=gt_mask[..., 0].squeeze(),
        pr_mask=pr_mask[..., 0].squeeze(),
    )

In [ ]:
# Try changing loss to sm.losses.bce_jaccard_loss
# Try changing backbone to resnet34 and resnet50
# Try dividing the data into different classes and segment each class separately
# Change batch_size and change learning_rate to 1e-05